In [ ]:
import json

import numpy as np
import pandas as pd
from mlcbakery import bakery_client
import os
client = bakery_client.Client("http://localhost:8081", token=os.getenv("ADMIN_AUTH_TOKEN"))

def add_rand_column_to_dataset(path_to_csv, column_name):
    df = pd.read_csv(path_to_csv)
    df[column_name] = np.random.randint(0, 100)
    df.to_csv(path_to_csv, index=False)

def load_json(path):
    with open(path, "r") as f:
        return json.load(f)
    

collection_name = "test_provenance"
client.find_or_create_by_collection_name(collection_name)
client.update_collection_storage_info(collection_name, storage_info=load_json(os.getenv("GCP_SA_PATH")), storage_provider="gcp")

In [ ]:
# prepares the dataset folder for the bakery by creating a .manifest.json file based on the folder structure
client.prepare_dataset("../test_datasets/doors", "doors", collection_name)

In [ ]:
client.save_to_bakery("../test_datasets/doors", upload_data=True)

In [ ]:
client.duplicate_dataset("../test_datasets/doors", "../test_datasets/doors.updated", {
    "properties": {
        "name": "doors.updated"
    }
})

In [ ]:
client.save_to_bakery("../test_datasets/doors.updated", upload_data=False)

In [ ]:
client.get_upstream_entities(collection_name, "doors.updated")

In [ ]:
# source = "D1_002"
# target = "D1_003"
# try:
#     client.duplicate_dataset(f"datasets/{source}", f"datasets/{target}", params={
#         "properties": {
#             "name": target
#         },
#     })
# except Exception as e:
#     print(e)
#     pass
# add_rand_column_to_dataset(f"datasets/{target}/data/data.csv", target)
# client.save_to_bakery(f"./datasets/{target}", upload_data=True)
# client.create_entity_relationship("dataset/test25/D1_001", "generated", "dataset/test25/D1_000")